In [1]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pickle
import json
import datetime

In [30]:
def crawlPage(url):
    
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text,'lxml')

    ## 去除非文字部分
        #player-and-links
        #infgraphicsAttach
    for push in soup.select("span"):
        push.extract()
    
    for push in soup.select("div.wsw > div"):
        push.extract()
      
    contents = soup.select('div.wsw')

    articles_dict = {}

    for page in range(len(contents)):
    
        articles_dict[page+1] = contents[page].text.replace('\n', '').replace('\'', '').replace('\"', '').strip()
    
    return articles_dict

In [4]:
f = open('e://voaUrl/_599_2016_url.txt','rt')
lines = f.readlines()
  
for i in lines:
    html = urlopen(i)

    soup = BeautifulSoup(html, 'lxml')
        
    articles_dict = {}
    
    url = 'http://www.voanews.com'
    
    #日期
    for date in soup.select('span.date'):
        
        articles_dict['date'] = datetime.datetime.strptime(date.text, "%B %d, %Y")

    for usa in soup.select('#ordinaryItems > li > div'):
    
        links = usa.select_one('a')['href']
    
        articles_dict['title'] = usa.select_one('a')['title']
        
        articles_dict['source'] = 'VOA'
        
        articles_dict['links'] = url + links
    
        articles_dict['contents'] = crawlPage(url + links)
    
        #print(articles_dict)
        
        # 轉換日期為json格式
        def date_handler(obj):
            return obj.date().isoformat() if hasattr(obj, 'isoformat') else obj

        jsObj = json.dumps(articles_dict,ensure_ascii=False,default=date_handler)
        newsText = open('e://voaUrl/test_contents.txt','a',encoding='utf-8')
        newsText.write(jsObj)  
        newsText.close() 

'http://www.voanews.com/a/ground-zero-now-and-then/1771867.html'

In [7]:
def crawlPage(url):
    
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text,'lxml')

    for push in soup.select("span"):
        push.extract()
    
    for push in soup.select("div.wsw > div"):
        push.extract()
    
    try:  
        contents = soup.find('div',{'class':'wsw'}).get_text().replace('\n', '').replace('\'', '').replace('\"', '').strip().split('. ')
    except:
        try: 
            contents = soup.find('div',{'class':'intro'}).get_text().replace('\n', '').replace('\'', '').replace('\"', '').strip().split('. ')
        except:
            contents = {}
    
    articles_dict = {}

    for page in range(len(contents)):
    
        articles_dict[str(page+1)] = contents[page]
    
    return articles_dict

In [8]:
crawlPage('http://www.voanews.com/a/fire-injures-one-chicago-hancock-skyscraper/3068332.html')

{'1': 'A fire that broke out Saturday halfway up the John Hancock Center, one of Chicagos tallest skyscrapers, has been put out, authorities said',
 '2': 'Five people were injured.Flames and thick smoke poured out of an east-side window on the 50th floor',
 '3': 'Authorities said the fire did not leave the residential unit where it began, but the apartment was gutted and other residents on the floor were displaced.The Chicago Fire Department said the blaze was accidental.The iconic 99-floor building, completed in 1970, also has offices, shops and an observation floor popular with tourists.The fire departments media office tweeted that of the injured, two were in good condition and three were stable',
 '4': 'A spokesman couldnt immediately be reached to provide details.Firefighters conducted floor-by-floor searches to ensure people were safe.'}